## Telco Customer Churn Data Cleaning

In [8]:
## This notebook cleans the Telco Customer Churn dataset for analysis.

# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv(r"C:\Users\Motun\Downloads\WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Initial exploration
print("Dataset shape:", df.shape)
print("\nFirst 5 rows:")
display(df.head())
print("\nData types and missing values:")
display(df.info())
print("\nSummary statistics:")
display(df.describe(include='all'))


# ## Step 1: Handle Missing Values


# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())


# Handle the missing TotalCharges (found in initial exploration)
# Convert TotalCharges to numeric, coerce errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')


# Check how many missing values we have in TotalCharges
print("\nMissing values in TotalCharges:", df['TotalCharges'].isnull().sum())


# Investigate rows with missing TotalCharges
display(df[df['TotalCharges'].isnull()])


# Since these customers have tenure=0, we can assume TotalCharges should be 0
df['TotalCharges'] = df['TotalCharges'].fillna(0)


# Verify no more missing values
print("\nMissing values after cleaning:")
print(df.isnull().sum())


# ## Convert Data Types

# Convert SeniorCitizen from 0/1 to categorical
df['SeniorCitizen'] = df['SeniorCitizen'].map({0: 'No', 1: 'Yes'})


# Convert categorical columns to category dtype
categorical_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 
                   'PhoneService', 'MultipleLines', 'InternetService',
                   'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                   'TechSupport', 'StreamingTV', 'StreamingMovies',
                   'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']

for col in categorical_cols:
    df[col] = df[col].astype('category')


# Convert TotalCharges to float
df['TotalCharges'] = df['TotalCharges'].astype(float)


# Verify data types
print("\nData types after conversion:")
display(df.dtypes)

# ## Step 3: Handle Inconsistent Values

# Check for inconsistent values in categorical columns
for col in categorical_cols:
    print(f"\nUnique values in {col}:")
    print(df[col].unique())


# Replace "No phone service" and "No internet service" with "No" for consistency
services = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 
            'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

for col in services:
    df[col] = df[col].astype(str).replace({
    'No phone service': 'No',
    'No internet service': 'No'
    }).astype('category')



# Verify changes
print("\nAfter cleaning service columns:")
display(df[services].head())


# ## Step 4: Create New Features (Optional)

# Optionally create new features that might be useful for analysis
# For example, total services subscribed to
service_columns = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
                  'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

# Count of services (excluding InternetService which is a type)
df['TotalServices'] = (df[service_columns[1:]] == 'Yes').sum(axis=1)

# Monthly charge per service
df['ChargePerService'] = np.where(df['TotalServices'] > 0, 
                                 df['MonthlyCharges'] / df['TotalServices'], 
                                 0)


# Display the new features
display(df[['customerID', 'TotalServices', 'ChargePerService']].head())


# Final Data Quality Check

# Check for any remaining issues
print("\nFinal data quality check:")
print("Duplicate rows:", df.duplicated().sum())
print("\nData types:")
display(df.dtypes)
print("\nSummary statistics:")
display(df.describe(include='all'))

# Save 

# Save the cleaned dataset to a new CSV file
df.to_csv('Telco_Customer_Churn_Cleaned.csv', index=False)

Dataset shape: (7043, 21)

First 5 rows:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes



Data types and missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessB

None


Summary statistics:


...

,customerID,TotalServices,ChargePerService
0,7590-VHVEG,1,29.850
1,5575-GNVDE,2,28.475
2,3668-QPYBK,2,26.925
3,7795-CFOCW,3,14.100
4,9237-HQITU,0,0.000



Final data quality check:
Duplicate rows: 0

Data types:


customerID            object
gender              category
SeniorCitizen       category
Partner             category
Dependents          category
tenure                 int64
PhoneService        category
MultipleLines       category
InternetService     category
OnlineSecurity      category
OnlineBackup        category
DeviceProtection    category
TechSupport         category
StreamingTV         category
StreamingMovies     category
Contract            category
PaperlessBilling    category
PaymentMethod       category
MonthlyCharges       float64
TotalCharges         float64
Churn               category
TotalServices          int64
ChargePerService     float64
dtype: object


Summary statistics:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,TotalServices,ChargePerService
count,7043,7043,7043,7043,7043,7043.000000,7043,7043,7043,7043,...,7043,7043,7043,7043,7043,7043.000000,7043.000000,7043,7043.000000,7043.000000
unique,7043,2,2,2,2,NaN,2,2,3,2,...,2,2,3,2,4,NaN,NaN,2,NaN,NaN
top,7590-VHVEG,Male,No,No,No,NaN,Yes,No,Fiber optic,No,...,No,No,Month-to-month,Yes,Electronic check,NaN,NaN,No,NaN,NaN
freq,1,3555,5901,3641,4933,NaN,6361,4072,3096,5024,...,4336,4311,3875,4171,2365,NaN,NaN,5174,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,32.371149,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,64.761692,2279.734304,NaN,2.459747,22.619814
std,NaN,NaN,NaN,NaN,NaN,24.559481,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30.090047,2266.794470,NaN,2.045539,19.406747
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18.250000,0.000000,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,35.500000,398.550000,NaN,1.000000,12.066250
50%,NaN,NaN,NaN,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,70.350000,1394.550000,NaN,2.000000,20.810000
75%,NaN,NaN,NaN,NaN,NaN,55.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,89.850000,3786.600000,NaN,4.000000,29.729167
